# 01 - Transfer Feature Releases Unit

**Purpose**: Extract Microsoft Fabric feature releases from Microsoft Learn what's new documentation

**Inputs**:
- Microsoft Learn URL: https://learn.microsoft.com/en-us/fabric/get-started/whats-new

**Outputs**:
- Delta Table: `feature_releases`

**Frequency**: Daily (recommended)

In [ ]:
import requests
import re
from datetime import datetime, timedelta
from pyspark.sql import functions as F
from pyspark.sql.types import *

## Parameters

In [ ]:
# Parameters (can be passed from Pipeline)
whats_new_url = "https://learn.microsoft.com/en-us/fabric/get-started/whats-new"
lookback_days = 90  # Extract features from last 90 days

## Step 1: Fetch What's New Content

In [ ]:
print(f"🔄 Fetching Fabric What's New from: {whats_new_url}")

try:
    response = requests.get(whats_new_url, timeout=30)
    response.raise_for_status()
    content = response.text
    print(f"✅ Successfully fetched {len(content)} characters")
except Exception as e:
    print(f"❌ Error fetching content: {e}")
    raise

## Step 2: Parse Feature Releases

In [ ]:
def parse_feature_releases(html_content):
    """
    Parse feature releases from Microsoft Learn What's New page
    Returns list of dicts with feature information
    """
    features = []
    
    # Pattern to match feature sections
    # Example: ## September 2024
    month_pattern = r'##\s+([A-Za-z]+\s+\d{4})'
    
    # Pattern to match feature entries
    # Example: ### Data Warehouse - New feature (Preview)
    feature_pattern = r'###\s+(.+?)\s*(?:\(([^)]+)\))?'
    
    # Split content by months
    months = re.split(month_pattern, html_content)
    
    for i in range(1, len(months), 2):
        if i+1 >= len(months):
            break
            
        month_str = months[i].strip()
        month_content = months[i+1]
        
        try:
            release_date = datetime.strptime(month_str, "%B %Y")
        except:
            continue
        
        # Find all features in this month
        feature_matches = re.finditer(feature_pattern, month_content)
        
        for match in feature_matches:
            feature_name = match.group(1).strip()
            status = match.group(2).strip() if match.group(2) else "GA"
            
            # Determine workload from feature name
            workload = "Unknown"
            if any(kw in feature_name.lower() for kw in ["data warehouse", "warehouse", "sql"]):
                workload = "Data Warehouse"
            elif any(kw in feature_name.lower() for kw in ["data factory", "pipeline", "dataflow"]):
                workload = "Data Factory"
            elif any(kw in feature_name.lower() for kw in ["power bi", "semantic model", "report"]):
                workload = "Power BI"
            elif any(kw in feature_name.lower() for kw in ["real-time", "kql", "eventhouse"]):
                workload = "Real-Time Intelligence"
            elif any(kw in feature_name.lower() for kw in ["lakehouse", "onelake"]):
                workload = "Data Engineering"
            elif any(kw in feature_name.lower() for kw in ["data science", "ml", "notebook"]):
                workload = "Data Science"
            
            # Determine if it's a preview feature
            is_preview = "preview" in status.lower()
            
            features.append({
                "feature_id": f"{release_date.strftime('%Y%m')}_{hash(feature_name) % 100000}",
                "feature_name": feature_name,
                "workload": workload,
                "release_date": release_date,
                "status": status,
                "is_preview": is_preview,
                "source_url": whats_new_url,
                "extracted_date": datetime.now()
            })
    
    return features

In [ ]:
print("🔄 Parsing feature releases...")
features_data = parse_feature_releases(content)

# Filter to last N days
cutoff_date = datetime.now() - timedelta(days=lookback_days)
features_data = [f for f in features_data if f["release_date"] >= cutoff_date]

print(f"✅ Parsed {len(features_data)} feature releases from last {lookback_days} days")

# Preview
if features_data:
    print("\n📋 Sample features:")
    for feat in features_data[:3]:
        print(f"  - {feat['feature_name']} ({feat['status']}) - {feat['workload']}")

## Step 3: Create DataFrame

In [ ]:
schema = StructType([
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("workload", StringType(), True),
    StructField("release_date", TimestampType(), False),
    StructField("status", StringType(), True),
    StructField("is_preview", BooleanType(), False),
    StructField("source_url", StringType(), True),
    StructField("extracted_date", TimestampType(), False)
])

df_features = spark.createDataFrame(features_data, schema=schema)

print(f"✅ Created DataFrame with {df_features.count()} rows")
df_features.show(5, truncate=False)

## Step 4: Write to Delta Lake

In [ ]:
# Define table path
table_name = "feature_releases"
table_path = f"Tables/{table_name}"

print(f"🔄 Writing to Delta table: {table_name}")

# Write with merge logic (upsert)
try:
    from delta.tables import DeltaTable
    
    # Check if table exists
    if DeltaTable.isDeltaTable(spark, table_path):
        print("  → Table exists, performing MERGE (upsert)...")
        
        delta_table = DeltaTable.forPath(spark, table_path)
        
        # Merge: Update existing, insert new
        delta_table.alias("target").merge(
            df_features.alias("source"),
            "target.feature_id = source.feature_id"
        ).whenMatchedUpdateAll(
        ).whenNotMatchedInsertAll(
        ).execute()
        
        print("  ✅ MERGE completed")
    else:
        print("  → Table doesn't exist, creating new table...")
        df_features.write.format("delta").mode("overwrite").save(table_path)
        print("  ✅ Table created")
    
    # Show final count
    final_count = spark.read.format("delta").load(table_path).count()
    print(f"\n✅ Total records in {table_name}: {final_count}")
    
except Exception as e:
    print(f"❌ Error writing to Delta: {e}")
    raise

## Step 5: Summary Statistics

In [ ]:
print("\n📊 Feature Release Statistics:")
print("=" * 60)

df_summary = spark.read.format("delta").load(table_path)

# By workload
print("\n🔸 By Workload:")
df_summary.groupBy("workload").count().orderBy(F.desc("count")).show(truncate=False)

# By status
print("\n🔸 By Status:")
df_summary.groupBy("status").count().orderBy(F.desc("count")).show(truncate=False)

# Preview vs GA
print("\n🔸 Preview vs GA:")
df_summary.groupBy("is_preview").count().show(truncate=False)

# Recent releases (last 30 days)
recent_cutoff = datetime.now() - timedelta(days=30)
recent_count = df_summary.filter(F.col("release_date") >= recent_cutoff).count()
print(f"\n🔸 Features released in last 30 days: {recent_count}")

print("\n" + "=" * 60)
print("✅ Transfer Feature Releases Unit - COMPLETED")